# TravelAIAgent: Overview ℹ️
Author: Ramis Hasanli  
[LinkedIn](https://www.linkedin.com/in/hasanliramis/)

🙂 Welcome to my submission for the 5-day **Google x Kaggle Generative AI Capstone Project!**

## **TravelAIAgent: A Multimodal Conversational Travel Planner**

In this project, we build **TravelAIAgent**, a multimodal conversational assistant that helps users plan and enjoy their trips using natural language, photos, and real-time data. It combines intelligent itinerary building, contextual cultural advice, event lookup, photo translation, and user preferences — all in a notebook-friendly chatbot you can interact with directly. Just scroll to the bottom to try it out!

---

## 🧠 What Problem Does It Solve?

Travelers often bounce between weather apps, ticketing sites, translation tools, and tourist blogs — especially when on the go.

**TravelAIAgent simplifies this by acting like a smart, friendly AI travel companion** who:
- Understands both **natural language** and **tourist photos**
- Offers **personalized suggestions** based on your preferences
- Uses **real-time APIs** for weather and events
- Generates **multi-day travel itineraries**
- Evaluates its own recommendations with an LLM “meta-check”
- Summarizes and translates **text in images**
- Stores helpful memory like your favorite activities or last city

---

## ⚙️ How It Works: Architecture & Core Design

The assistant is built around an **intent-based routing system** that detects what kind of request you're making, and sends it to a specific handler.

### 🔧 Key Components:
- **Gemini 2.0 Flash + Gemini 1.5 Pro Vision** for all language and image understanding
- **OpenWeather API** for live and forecasted weather data
- **Ticketmaster Discovery API** for city events and experiences
- **ChromaDB + `text-embedding-004`** to deliver grounded cultural tips (RAG-style)
- **Python CLI Chat Loop** that mimics real chatbot experience in a Kaggle Notebook
- **Show Markdown Renderer** that formats all AI responses in readable, friendly Markdown
- **Modular Intent Handlers** like `handle_get_weather`, `handle_plan_itinerary`, etc.
- **Session Memory** for last city and user travel preferences
- **LLM-Based Evaluation System** that gives each itinerary a “Travel Score” and feedback on weather fit, personalization, balance, and authenticity

---

## ✨ Functionality

TravelAIAgent supports a wide range of features, including:

- **🌦 Weather Info** — current and 5-day forecast using Gemini summaries
- **🧳 Travel & Cultural Tips** — grounded tips from 30+ cities via vector search + RAG
- **🎟 Event Discovery** — find what's happening near you using Ticketmaster API
- **🗺 Itinerary Generator** — builds personalized plans using your preferences, weather, and events
- **🧠 Itinerary Evaluator (Gen AI)** — TravelAIAgent self-assesses its own plans with a “🌍 Travel Score” out of 10
- **🖼 Landmark Descriptions** — upload a photo and get a tourist-style explanation
- **📸 OCR & Translation** — extract and translate any text from images (e.g., signs, menus)
- **💾 Memory** — remembers your likes and city history to make smarter suggestions
- **📝 Export Itineraries** — choose to save as `.md` or `.pdf` right after generation
- **📤 Export Full Chat History** — just type `!export` anytime

---

## Notes

- 🛠 **Setup required:** Enter your API keys and run all setup cells first.
- 📁 **Photo processing:** Attach a Kaggle dataset named `photos` for image features to work. Upload photos in .png format. To use photos, reference them by name like "Hey AI, can you translate the text you see in fileName.png?", "What do I see on fileName.png?", etc.
- ✨ **Export:** Use `!export` to save full chat history or export itineraries post-generation.
- ❌ **Exit options:** Use `!quit`, `!q`, or `quit` to end the chat.

Enjoy your trip planning with TravelAIAgent in the last section of the notebook named **"Chat with TravelAIAgent"** 🌍✈️

# TravelAIAgent: Setup 🛠️
First, install ChromaDB and the Gemini API Python SDK. Then, import all necessary Python dependecies.

In [1]:
!pip uninstall -qqy jupyterlab kfp
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3" "reportlab"

In [2]:
from google import genai
from google.genai import types
from google.api_core import retry
from IPython.display import HTML, Markdown, display
import chromadb
from chromadb.utils.embedding_functions import EmbeddingFunction
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

### Automated retry

In [3]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry
from google.api_core import retry
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## 🔑 API keys

To run the following cell, your API keys must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`, `WEATHER_API_KEY`, and `EVENTS_API_KEY`.\
GOOGLE_API_KEY - Gemini API Key.\
WEATHER_API_KEY - OpenWeather API Key.\
EVENTS_API_KEY - TicketMaster Discovery API Key.\
To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret('GOOGLE_API_KEY')
WEATHER_API_KEY = user_secrets.get_secret('WEATHER_API_KEY')
EVENTS_API_KEY = user_secrets.get_secret('EVENTS_API_KEY')

In [5]:
TRAVELAGENT_SYSINT = (
    "You are TravelAIAgent, a helpful and friendly AI travel companion. You're chatting with someone who is planning or enjoying a trip. "
    "Your role is to offer smart, practical advice in a natural, engaging tone — like a well-traveled friend or human travel planner. "
    "You assist with weather forecasts, cultural tips, local events, itinerary planning, image/text translation, and tourist insights. "
    "You remember preferences the user shares (like what they enjoy or dislike) and use that to personalize your suggestions. "
    "Be relaxed and human-sounding. Avoid robotic or overly formal language. Use emojis only when they help convey emotion or friendliness. "
    "Offer helpful suggestions based on what the user says — but don't overload them with too much info unless they ask for it."
)

onboarding_message = (
    "👋 Hi there! I'm TravelAIAgent — your AI-powered companion for smarter travel planning 🌍✈️\n\n"
    "Here’s what I can help you with:\n"
    "- 🌦️ Get real-time weather info and multi-day forecasts for any city\n"
    "- 🧳 Share cultural tips, local etiquette, and safety advice\n"
    "- 🌐 Translate short phrases or signs into English\n"
    "- 📸 Read and translate text from uploaded travel photos\n"
    "- 🏛️ Describe and answer questions about landmarks in your photos\n"
    "- 🎟️ Find events, concerts, and things to do during your trip\n"
    "- 🗺️ Build a personalized day-by-day travel itinerary using your preferences, the weather, and local events\n\n"
    "🙂 Just talk to me naturally — what's on your mind? What are your travel preferences? Do you have any travel plans soon?"
)

# This block connects the assistant to the Gemini API from Google and sets up the generation behavior. We're using:
# temperature = 0.7 for a balanced level of creativity and stability — just enough flair for conversational responses.
# max_output_tokens = 512 to limit overly long replies.
# system_instruction = TRAVELAGENT_SYSINT to ensure every message stays in character as a smart, fun travel companion.
# This configuration is used in every Gemini content generation call, shaping how the assistant responds to user input throughout the entire experience.
client = genai.Client(api_key=GOOGLE_API_KEY)
config = types.GenerateContentConfig(
    temperature = 0.7,
    max_output_tokens = 512,
    system_instruction = TRAVELAGENT_SYSINT,
)

# TravelAIAgent: Backend 🧑‍💻

## 🧠 Interpreting User Intent
One of the most important parts of TravelAIAgent is understanding what the user is actually asking for — whether they want the weather, cultural tips, a travel itinerary, or help with a photo. To handle this, we use a technique called function routing, powered by Gemini.

The interpret_user_request() function acts as the brain of this routing system. It sends the user's message to Gemini with a carefully crafted prompt that explains how to classify the input. The prompt includes examples of how to convert natural language into a structured Python dictionary, such as: {'intent': 'get_weather', 'location': 'Paris'} or {'intent': 'plan_itinerary', 'location': 'Rome', 'days': 3, 'when': 'next week'} or {'intent': 'get_tip', 'query': 'public transport in Tokyo'}

Gemini then replies with a dictionary string, and we use Python’s ast.literal_eval() to safely convert that string into a real Python dictionary. This dictionary tells the assistant which intent to handle, and which handler function to trigger.
If something goes wrong — such as Gemini returning an invalid dictionary — we default to a generic 'chat' intent, so the assistant can still respond gracefully.

---

## 🏙️ Extracting City Names
Sometimes users don’t explicitly state a city in a clean format, especially when asking for travel tips or events. To improve contextual understanding, we use the extract_city_from_text() function. This is a lightweight utility that asks Gemini to pull a city name from any given sentence.

For example:
“Tell me more about Zurich” → returns "Zurich", “What can I do there?” → returns "None"

It helps ensure the assistant tracks which city is being discussed, so it can recall that later (e.g. when asked "What's the weather like there?"). This adds a subtle layer of memory and personalization to the interaction, without requiring complex NER models or external parsers.

In [6]:
# BETA
# Show response
from IPython.display import display, Markdown, Image

def show_response(response, label="🤖 TravelAIAgent said"):
    # Show label as Markdown header
    if label:
        display(Markdown(f"**{label}:**"))

    for p in response.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())

    display(Markdown('----'))


In [7]:
import ast

def interpret_user_request(user_input):
    prompt = (
        "You are a function router. Based on the user message, output ONLY a Python dictionary.\n"
        "- If the user is asking for weather info, return: {'intent': 'get_weather', 'location': '<CITY>'}\n"
        "- If the user is asking for tips or local info (scams, etiquette, transport, etc.), return: {'intent': 'get_tip', 'query': '<QUESTION>'}\n"
        "- If the user asks about the weather in last mentioned city, return: {'intent': 'get_weather_last'}\n"
        "- If the user asks what city they last mentioned, return: {'intent': 'get_last_city'}\n"
        "- If the user asks to read and translate a photo, return: {'intent': 'image_translate', 'filename': '<FILENAME>'}\n"
        "- If the user asks what is shown in a specific photo, return: {'intent': 'describe_photo', 'filename': '<FILENAME>'}\n"
        "- If the user asks a question about a previously uploaded photo, return: {'intent': 'ask_about_photo', 'question': '<QUESTION>'}\n"
        "- If the user asks for a travel plan or itinerary, return: {'intent': 'plan_itinerary', 'location': '<CITY>', 'days': <NUMBER>, 'when': '<TIMEFRAME>'}\n"
        "- If the user asks for weather in the future (e.g. weather next week, forecast for the next few days), return: {'intent': 'get_weather_forecast', 'location': '<CITY>'}\n"
        "- If the user asks for upcoming events or things to do in a city, return: {'intent': 'get_events', 'location': '<CITY>', 'when': '<TIMEFRAME>'}\n"
        "- Otherwise, return: {'intent': 'chat'}\n"
        "Respond with ONLY the dictionary. No explanations, no code blocks.\n\n"
        f"User: {user_input}"
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    try:
        return ast.literal_eval(response.text.strip())
    except Exception:
        return {'intent': 'chat'}


def extract_city_from_text(user_input: str) -> str | None:
    prompt = (
        "You are a city name extractor. If the following sentence clearly refers to a known city, "
        "output ONLY the city name as a string, with no extra text.\n"
        "If no city is clearly mentioned, output 'None'.\n\n"
        f"Sentence: {user_input}"
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    result = response.text.strip().strip("'\"")
    return None if result.lower() == "none" else result



## 🌦️ Real-Time & Forecasted Weather
TravelAIAgent helps users make smart decisions while traveling — and weather is a big part of that! These two functions (get_weather and get_weather_summary) fetch and summarize weather data in a friendly, travel-savvy tone using a combination of the OpenWeather API and Gemini 2.0 Flash.

**get_weather(city) – Current Conditions**\
This function fetches the real-time weather for a given city using the OpenWeather API. Instead of just dumping raw data, it prepares a natural-language summary with the help of Gemini. The API response is parsed to extract key weather details like: Current temperature, Feels-like temperature, Humidity, Wind speed, General weather description. Then, a prompt is crafted to instruct Gemini to summarize this like a local travel agent would — conversational, helpful, and using emojis where appropriate.\
For example:
“It’s around 15°C in Lisbon right now with a nice breeze and clear skies — perfect for walking the old town! 🌤️👟”
This makes the response feel more human and trip-relevant, not just like reading numbers.

**get_weather_summary(city) – 5-Day Forecast**\
Instead of a snapshot, this function gathers a multi-day forecast from OpenWeather, broken into 3-hour segments across five days. The forecast entries are grouped by date, and then passed to Gemini in a structured format. The LLM is asked to: Summarize the general vibe of each day, highlight rain or great outdoor weather, and recommend the best days for activities. This is especially useful when building itineraries, since it helps the assistant match sunny days with walking tours or outdoor activities, and reserve indoor options for rainy ones.\
For example:
“Tuesday looks rainy 🌧️ — maybe plan some museum time. But Wednesday through Friday look warm and mostly sunny — great for beach walks or sightseeing! ☀️🌊”

In [8]:
# Get weather info
import requests
import datetime

# Current weather
def get_weather(city):
    url = (
        f"http://api.openweathermap.org/data/2.5/weather?"
        f"q={city}&appid={WEATHER_API_KEY}&units=metric"
    )
    response = requests.get(url)
    if response.status_code != 200:
        return f"Sorry, I couldn’t fetch the weather for {city}."

    data = response.json()
    description = data["weather"][0]["description"]
    temp = data["main"]["temp"]
    feels_like = data["main"]["feels_like"]
    humidity = data["main"].get("humidity", "N/A")
    wind_speed = data["wind"].get("speed", "N/A")

    # Create input for Gemini summarization
    weather_text = (
        f"City: {city}\n"
        f"Description: {description}\n"
        f"Temperature: {temp}°C (feels like {feels_like}°C)\n"
        f"Humidity: {humidity}%\n"
        f"Wind Speed: {wind_speed} m/s"
    )

    prompt = (
    "You are a friendly travel assistant. Provide a casual and helpful weather update using the info below. "
    "Do not include any greetings like 'Hey there' or 'Hi'. "
    "Start directly with the weather description. Use emojis where appropriate to enhance clarity and friendliness, but keep it concise and useful.\n\n"
    f"{weather_text}"
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    return response.text.strip()


# Future forecast
def get_weather_summary(city: str) -> str:
    url = (
        f"http://api.openweathermap.org/data/2.5/forecast?"
        f"q={city}&appid={WEATHER_API_KEY}&units=metric"
    )
    response = requests.get(url)
    if response.status_code != 200:
        return f"Sorry, I couldn’t fetch the forecast for {city}."

    data = response.json()

    # Group by date
    forecasts_by_day = {}
    for entry in data["list"]:
        dt = datetime.datetime.fromtimestamp(entry["dt"])
        date_str = dt.date().isoformat()
        time_str = dt.strftime("%H:%M")
        desc = entry["weather"][0]["description"]
        temp = entry["main"]["temp"]

        if date_str not in forecasts_by_day:
            forecasts_by_day[date_str] = []
        forecasts_by_day[date_str].append(f"{time_str}: {desc}, {temp:.1f}°C")

    # Prepare text for Gemini to summarize
    forecast_text = f"City: {city}\nForecast for the next 5 days:\n\n"
    for date, slots in forecasts_by_day.items():
        forecast_text += f"Date: {date}\n" + "\n".join(f"  - {slot}" for slot in slots) + "\n\n"

    # Ask Gemini to summarize it in a friendly way
    prompt = (
    f"You are a helpful travel assistant. Write a casual, helpful summary of this 5-day weather forecast in {city}. "
    f"Do not include greetings or introductions like 'Hey there'. Just focus on summarizing the weather in a friendly way. "
    f"Highlight which days are great for exploring. Use emojis sparingly to illustrate weather patterns.\n\n{forecast_text}"
    )
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    return response.text.strip()

## 🖼️ Understanding & Translating Travel Photos

Travelers often encounter signs, menus, landmarks, and cultural artifacts they don’t fully understand. These three functions add powerful image understanding capabilities to TravelAIAgent by leveraging Gemini 1.5 Pro Vision for visual tasks and Gemini 2.0 Flash for language translation. The result is a travel assistant that can read and interpret photos like a human guide would.\
**User can upload photos (.png) to Kaggle dataseet named "photos" and attach it to this notebook. This will allow Travel Agent to interact with photos. To reference the photo in chat, user needs to use followig format "imgName.png". For example: Translate the text from the picture RoseGarden.png**

**extract_text_from_image(file_path) – OCR with Gemini Vision** 📷🔤\
This function allows users to upload an image (like a street sign, museum label, or food menu), and have the assistant extract all visible text from it. It uses Gemini 1.5 Pro’s multimodal vision model, which is capable of reading embedded text from real-world photos.\
The function opens the image file in binary mode, attaches it to a prompt asking Gemini to extract all visible text (but not translate), and then returns the raw string. This step is useful when the goal is to understand what’s written in a local language before translating or explaining it.

**translate_to_english(text) – Text Translation** 🌐🗣️\
Once the text is extracted from an image, we pass it to this function, which uses Gemini 2.0 Flash to translate it into English. The prompt is simple and direct: translate the following text.\
This lets users take photos of signs, menus, schedules, or maps and instantly understand what they mean — perfect for tourists navigating unfamiliar environments.\
For example: A French street sign like "Rue des Bourneaux" → "Bourneaux Street"

**describe_photo(file_path) – Landmark and Scene Description** 🏛️✨\
This function goes beyond OCR and answers a higher-level question: “What am I looking at?”\
It uses Gemini 1.5 Pro to analyze the image and describe the scene, including its possible cultural or historical context. It’s especially useful for travel photos like landmarks, public art, or architecture. The prompt instructs Gemini to describe what’s in the image and why it might matter to a traveler — similar to how a museum guide or tour book would explain a location.\
For example: Upload a photo of the Eiffel Tower → “This is the Eiffel Tower in Paris, an iconic symbol of France built for the 1889 World's Fair…”

In [9]:
# Extract text from image
def extract_text_from_image(file_path: str) -> str:
    with open(file_path, "rb") as img:
        image_bytes = img.read()

    prompt = "Extract all visible text from this image. Don't translate it."

    response = client.models.generate_content(
        model="gemini-1.5-pro-latest",
        contents=[
            types.Part.from_text(text=prompt),
            types.Part.from_bytes(data=image_bytes, mime_type="image/png")
        ]
    )
    return response.text.strip()

# Translate extracted text using Geimini Flash
def translate_to_english(text: str) -> str:
    prompt = f"Translate the following to English:\n\n{text}"
    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )
    return response.text.strip()

def describe_photo(file_path: str) -> str:
    with open(file_path, "rb") as img:
        image_bytes = img.read()

    prompt = (
        "You are a travel assistant. Describe this photo in detail. "
        "Explain what is shown, its possible cultural or historical significance, "
        "and any information that could help a traveler understand what they’re looking at."
    )

    response = client.models.generate_content(
        model="models/gemini-1.5-pro-latest",
        contents=[
            types.Part.from_text(text=prompt),
            types.Part.from_bytes(data=image_bytes, mime_type="image/png")  # or image/png
        ]
    )

    return response.text.strip()

## 🎟️ Discovering Local Events
One of the most exciting travel upgrades in TravelAIAgent is its ability to suggest real events happening in your destination. Whether you’re into concerts, theater, or cultural exhibits, the assistant uses this function to bring your itinerary to life with up-to-date entertainment options — all powered by the Ticketmaster Discovery API and summarized naturally using Gemini.

**get_events(city, start_date, end_date, max_events) – Event Discovery + Summarization**\
This function starts by querying the Ticketmaster Discovery API, using the name of the city and optional start/end date filters. It returns a list of upcoming events sorted by date — concerts, sports, exhibitions, shows, and more. We limit the number of results using max_events (default is 5) to avoid overwhelming the user and focus on the most relevant options. For each event, we extract key info such as name, date, venue\
Instead of listing raw data directly to the user, the assistant passes the list to Gemini 2.0 Flash using a well-crafted prompt. Gemini then summarizes the events in a casual, friendly tone — just like a travel agent would.\
For example: “This week in San Francisco: balloon art exhibits, jazz at Birdland, and some Broadway action if you’re feeling theatrical. 🎷🎭”\
The prompt also nudges Gemini to recommend which events suit different travelers (families, art lovers, music fans), highlight variety and uniqueness, and keep the tone fun and helpful

In [10]:
def get_events(city: str, start_date: str = None, end_date: str = None, max_events: int = 5) -> str:
    url = "https://app.ticketmaster.com/discovery/v2/events.json"

    params = {
        "apikey": EVENTS_API_KEY,
        "city": city,
        "sort": "date,asc",
        "locale": "*",
    }

    if start_date:
        params["startDateTime"] = start_date
    if end_date:
        params["endDateTime"] = end_date

    response = requests.get(url, params=params)
    if response.status_code != 200:
        return f"Sorry, I couldn't fetch events for {city}."

    data = response.json()
    events = data.get("_embedded", {}).get("events", [])[:max_events]

    if not events:
        return f"No upcoming events found in {city}."

    # Prepare input for Gemini
    raw_event_lines = []
    for event in events:
        name = event.get("name", "Unnamed Event")
        date = event.get("dates", {}).get("start", {}).get("localDate", "Unknown Date")
        venue = event.get("_embedded", {}).get("venues", [{}])[0].get("name", "Unknown Venue")
        line = f"{name} on {date} at {venue}"
        raw_event_lines.append(line)

    event_input = "\n".join(raw_event_lines)

    # Gemini prompt
    prompt = (
    f"You are a helpful travel assistant creating a quick event guide for someone visiting {city}. "
    f"Use a relaxed, natural tone — no greetings like 'Hey there' or introductions. "
    f"Highlight 3–5 events that are good for travelers, based on their variety (music, sports, art, comedy, etc.). "
    f"For each one, mention why it's interesting or who might enjoy it (e.g., for music lovers, for families, for night owls, etc.). "
    f"Write like a local giving tips to a friend. Keep it concise and insightful.\n\n"
    f"Events:\n{event_input}"
)

    summary_response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    return summary_response.text.strip()


## 🔍 Local Insights & Landmark Knowledge
To give TravelAIAgent the ability to answer grounded, location-specific questions — like cultural etiquette, local dos and don'ts, or tips about a photo — we use Retrieval Augmented Generation (RAG). This section combines ChromaDB (a vector database), Gemini embeddings, and Gemini's language generation to bring real-world knowledge into the assistant's responses.

**🧳 documents – Curated Travel Tips**\
We start with a hardcoded list of helpful, city-specific cultural tips and travel advice. These act as a lightweight knowledge base. Each item is a short, practical sentence — perfect for answering questions like: “What should I know before going to Tokyo?”, “Is it okay to haggle in Marrakech?”

**🧠 GeminiEmbeddingFunction – Semantic Understanding**\
To search through these tips in a meaningful way, we embed both the tips and the user's queries using Gemini's text-embedding-004 model. This converts text into high-dimensional vectors that capture their semantic meaning. The class supports two modes: "retrieval_document" – for indexing passages and "retrieval_query" – for embedding the user's question. This ensures the model can retrieve relevant advice even if the user asks in a different phrasing than what’s stored.

**🗃️ chroma_client + chromaDB – Embedding Storage**\
We initialize a ChromaDB collection named "travel_tips", embed the list of tips, and store them by unique IDs. This allows the assistant to quickly search for tips that are most relevant to the user's question at runtime.

**❓ search_and_answer(user_question) – RAG Answering**\
When a user asks for tips or cultural context: The function switches to query mode -> Finds the top 2 semantically similar tips from ChromaDB -> Passes those into Gemini 2.0 Flash using a prompt that instructs it to generate a helpful, grounded answer. This pattern ensures that answers are both relevant and informed by the actual data — a key aspect of building trust in AI assistants. In case of missing suggestion for particular city, the function asks LLM to provide user with tips.

**🖼️ Image Insights: photo_collection + search_photo_insights()**\
We use the same RAG pipeline to store and query descriptions of user-uploaded photos (e.g. landmarks, statues, monuments). After describing a photo using Gemini Vision, the description is embedded and added to a separate collection (photo_insights). Then, if the user asks: “What is that building?” or “What’s the story behind this statue?”. We use search_photo_insights() to pull relevant image descriptions from the vector DB and let Gemini answer based on that context.

Together, these components turn your assistant into a context-aware, grounded travel expert. It can recall city-specific etiquette, offer photo-based insight, and explain things like a human guide — all while staying efficient and scalable. ✈️🧠📸

In [11]:
documents = [
    "In Bangkok, visit Chatuchak Market on the weekend and avoid tuk-tuk scams near tourist spots. Street food is exceptional—try mango sticky rice and pad thai. Use BTS Skytrain to avoid traffic.",
    "In Paris, always greet with 'Bonjour' before asking questions. Most museums are closed on Mondays. Book Eiffel Tower and Louvre tickets in advance. Avoid dining near major tourist spots for better food and prices.",
    "In London, public transport uses contactless cards. Tipping is not expected in pubs. Visit museums like the British Museum and Natural History Museum—they’re free. Pack for rain regardless of season.",
    "In Dubai, public displays of affection are discouraged. Dress modestly in public areas. Visit Burj Khalifa at sunset and try desert safaris. Alcohol is only served in licensed venues.",
    "In Singapore, chewing gum is banned. It’s one of the cleanest and safest cities globally. Hawker centers offer world-class cheap food—try chicken rice. Tap water is safe to drink.",
    "In Kuala Lumpur, try nasi lemak for breakfast. Petronas Towers are best viewed at sunset. Use Grab for rides instead of taxis. Shopping malls like Pavilion are top-rated for both locals and tourists.",
    "In New York City, walk fast, tip 15–20%, and explore boroughs beyond Manhattan. The subway runs 24/7 but can be confusing—use apps like Citymapper. Times Square is worth seeing once, but avoid eating there.",
    "In Istanbul, try local ferry rides and visit both European and Asian sides of the city. Explore the Grand Bazaar, Hagia Sophia, and enjoy Turkish tea culture. Carry cash for smaller shops.",
    "In Tokyo, avoid loud phone calls on trains. Convenience stores (konbini) have everything. Use IC cards like Suica for transit. Respect local customs—bow slightly when greeting.",
    "In Antalya, the old town (Kaleiçi) is walkable and filled with ancient Roman ruins. Visit nearby beaches like Lara and Konyaaltı. Summer gets hot—carry water and wear sunscreen.",
    "In Seoul, Korean street food markets like Gwangjang are must-visit. Public transport is efficient. Use T-Money cards for metro and buses. Don’t tip—it's not customary.",
    "In Rome, beware of pickpockets at popular landmarks. Tap water is drinkable. Visit the Colosseum and Vatican early. Restaurants often charge a 'coperto'—a cover charge.",
    "In Phuket, island tours are best booked locally. Avoid animal-based attractions. Visit Phi Phi Islands and Big Buddha. Roads can be dangerous—avoid renting scooters unless experienced.",
    "In Mecca, non-Muslims cannot enter the central holy area. Stay hydrated in the heat. Book hotels well in advance during Hajj and Ramadan. Respect local customs strictly.",
    "In Hong Kong, use the Octopus card for transport and street food. Don't miss Victoria Peak. Enjoy dim sum culture and visit outlying islands like Lantau. Be aware of political sensitivities.",
    "In Barcelona, be cautious of pickpockets on Las Ramblas. Tapas culture is big—dine late. Visit Sagrada Familia and Park Güell. Siesta hours may affect shop openings in afternoons.",
    "In Zurich, trains are punctual to the minute. Public water fountains offer safe drinking water. Switzerland is expensive—budget accordingly. Visit Lake Zurich and nearby mountains.",
    "In Cairo, tipping (baksheesh) is expected for most services. Visit the pyramids early to avoid crowds. Use Uber instead of taxis for safety and transparency. Dress modestly.",
    "In Sydney, sun protection is essential year-round. Use an Opal card for public transport. Bondi and Manly beaches are popular. Tap water is safe, and coffee culture is strong.",
    "In Marrakech, haggle respectfully in souks. Fridays are holy—many shops may close. Visit Jardin Majorelle and stay in a traditional riad. Dress modestly and be cautious with street guides.",
    "In Amsterdam, watch for bikes at crossings. Many museums require advance reservations. Try local foods like stroopwafels and herring. Public transport is efficient and bike-friendly.",
    "In Mexico City, avoid tap water. Street tacos are a must, but go where locals eat. Visit Frida Kahlo Museum and Teotihuacan pyramids. Altitude can affect visitors—hydrate well.",
    "In Athens, the Acropolis is best visited early morning. Tipping is appreciated but not required. Enjoy Greek tavernas and try souvlaki and moussaka. Avoid driving—traffic is chaotic.",
    "In Vancouver, use contactless or Compass Card for transit. Pack for rain, even in summer. Stanley Park and Granville Island are must-visits. Tap water is clean and cold year-round.",
    "In Buenos Aires, late dinners and tango shows are local staples. Beware of counterfeit currency. Palermo is a trendy neighborhood for food and nightlife. Learn basic Spanish phrases.",
    "In Prague, the Old Town is stunning but crowded—explore Žižkov and Letná for a local vibe. Czech beer is famous and cheap. Public transport is reliable—get a travel pass.",
    "In Vienna, classical concerts abound, but dress semi-formally. Public transport is safe and clean. Visit Schönbrunn Palace and try Sachertorte. Tap water comes straight from the Alps.",
    "In Cape Town, avoid walking after dark in certain areas. Visit Table Mountain on a clear day. Enjoy coastal drives like Chapman’s Peak. Check safety alerts before hiking or beach trips.",
    "In Lisbon, trams can get crowded—beware of pickpockets. Try pastel de nata from local bakeries. Visit Belém Tower and ride Tram 28. Hills are steep—wear good shoes.",
    "In Los Angeles, public transport is limited—consider renting a car. Tipping is standard at 20%. Visit Griffith Observatory, Venice Beach, and explore neighborhoods like Silver Lake and Santa Monica.",
    "In Orlando, most visit for theme parks like Disney World and Universal. Summer is hot and humid—stay hydrated. Lines can be long—use mobile apps to plan attractions.",
    "In Las Vegas, casinos and shows dominate the Strip. Stay indoors during midday heat. Tipping is expected everywhere. Off-strip spots often offer better value and local experiences.",
    "In Miami, visit South Beach for nightlife and Wynwood for street art. Sunscreen is a must. Tap water is safe, and many locals speak Spanish fluently.",
    "In San Francisco, wear layers due to rapid weather changes. Use MUNI and BART for transport. Avoid leaving valuables in cars. Visit Alcatraz and walk the Golden Gate Bridge.",
    "In Chicago, deep-dish pizza is iconic. The 'L' train system is easy to use. Visit Millennium Park and the Art Institute. Winters are freezing—dress in layers.",
    "In Washington D.C., museums are free and top-notch—visit the Smithsonian. Metro is clean but avoid peak hours. Security is tight near landmarks. Cherry blossom season is very popular.",
    "In Boston, Freedom Trail offers a walk through history. Public transport is called the T. Locals are passionate about sports—catch a Red Sox game if possible.",
    "In Honolulu, beaches like Waikiki are iconic. Respect native Hawaiian culture. Sunscreen with reef-safe ingredients is encouraged. Hiking trails offer scenic ocean views.",
    "In San Diego, the weather is almost always perfect. Visit Balboa Park and La Jolla Cove. Public beaches are clean and accessible. Try fish tacos from local spots.",
    "In New Orleans, music and food culture are unmatched. Visit the French Quarter but be cautious late at night. Try beignets and gumbo. Respect local traditions and celebrations."
]


In [12]:
# Gemini Embedding Function class
class GeminiEmbeddingFunction(EmbeddingFunction):
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input):
        task_type = "retrieval_document" if self.document_mode else "retrieval_query"
        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(task_type=task_type)
        )
        return [e.values for e in response.embeddings]

# Initialize ChromaDB
chroma_client = chromadb.Client()
DB_NAME = "travel_tips"
embed_fn = GeminiEmbeddingFunction()

db = chroma_client.get_or_create_collection(
    name=DB_NAME,
    embedding_function=embed_fn
)

# Store tips
db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

# Store image description with ChromaDB
photo_collection = chroma_client.get_or_create_collection(
    name="photo_insights",
    embedding_function=embed_fn
)

def add_photo_description_to_chromadb(photo_id: str, description: str):
    photo_collection.add(
        documents=[description],
        ids=[photo_id]
    )

# Search photo description with a question
def search_photo_insights(user_question: str, top_k: int = 2) -> str:
    embed_fn.document_mode = False  # use query embedding

    results = photo_collection.query(query_texts=[user_question], n_results=top_k)
    matches = results["documents"][0]

    prompt = (
        "You are a helpful travel assistant. Use the following descriptions to answer the user's question.\n\n"
        f"QUESTION: {user_question}\n"
    )

    for desc in matches:
        prompt += f"DESCRIPTION: {desc.strip()}\n"

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )
    return response.text.strip()

# Search and answer
def search_and_answer(user_question):
    embed_fn.document_mode = False

    result = db.query(query_texts=[user_question], n_results=2)
    passages = result.get("documents", [[]])[0]

    # Updated prompt: Always try to help, even if passages are weak
    prompt = (
        "You are a helpful, friendly travel assistant. Use the following travel tips **if they are relevant** to help answer the user's question.\n"
        "If they aren't relevant, ignore them and still do your best to give helpful travel advice using your own knowledge.\n\n"
        f"USER QUESTION: {user_question}\n"
    )

    for passage in passages:
        prompt += f"PASSAGE: {passage.strip()}\n"

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    return response.text.strip()


## 🧠 Modular Intent Handlers
To keep TravelAIAgent organized and extensible, every user intent is handled by a dedicated function — known as a “handler.” These functions make the assistant more maintainable and allow for clear separation of logic, especially when integrating multiple tools like weather APIs, image understanding, vector search, and LLM generation.\
Each handler performs three key tasks:
1. Executes a specific capability (like getting weather or generating an itinerary)
2. Updates the assistant’s memory if needed (e.g. storing the last mentioned city)
3. Prints and stores the response in the conversation history

---

**🌤️ handle_get_weather & handle_get_weather_forecast**\
These handlers fetch current conditions or a 5-day forecast from OpenWeather and summarize it using Gemini. They store the city as memory["last_city"] so the assistant can reference it later when the user says, “What’s the weather like there?”

**🧳 handle_get_tip**\
This pulls grounded cultural advice from ChromaDB using a RAG-style query. It also extracts and stores the city mentioned in the user’s tip-related question to maintain context.

**📸 handle_describe_photo, handle_ask_about_photo, handle_image_translate**\
These three handlers handle photo-related requests:
1. describe_photo: Uses Gemini Vision to describe what’s in an uploaded image (e.g. a statue or landmark)
2. ask_about_photo: Allows follow-up questions using photo-based RAG from ChromaDB
3. image_translate: Extracts and translates visible text from a photo (e.g. street signs or menus)
They help users understand and interact with their surroundings more easily, especially in unfamiliar places.

**🌆 handle_get_events**\
This fetches events in a city using the Ticketmaster Discovery API and summarizes them using Gemini. The results feel natural and tailored — great for finding things to do on the fly.

**🧠 handle_get_weather_last & handle_get_last_city**\
These support context-awareness. If the user asks: “What’s the weather like there?” or “Which city was I talking about?”. The assistant responds intelligently based on the memory dictionary (memory["last_city"]), maintaining a natural and coherent conversation flow.

**🗺️ handle_plan_itinerary**

This is one of the most impressive handlers — it builds a personalized, multi-day itinerary by combining:

1. 🌦️ Real-time multi-day **weather forecasts**
2. 🧳 Cultural **travel tips** retrieved using **ChromaDB + Gemini embeddings**
3. 💬 **User preferences** (gathered naturally during conversation and stored in memory)
4. 🎟️ Local **event listings** using the Ticketmaster Discovery API

The handler intelligently estimates the trip start date based on natural language like "next week" or "this weekend", and then assembles all relevant information into a prompt that Gemini uses to generate a highly tailored itinerary.

---

🧠 **LLM-Based Self-Evaluation**: After the itinerary is generated, TravelAIAgent passes it back to Gemini for **self-assessment**. The model scores the itinerary on:
- Balance between indoor/outdoor activities
- Cultural authenticity
- Weather suitability
- Personalization based on your preferences

Gemini then responds with a 🌍 **Travel Score out of 10**, plus helpful feedback. This makes it easy for users to assess the quality of the plan, catch gaps, and get suggestions — making the experience more trustworthy and intelligent.

✨ **Export Features**: After generating the itinerary, the user is automatically asked whether they'd like to export it. If yes, the plan and evaluation can be saved in:
- 📄 **Markdown format (.md)**
- 📄 **PDF format** (beautifully rendered with basic formatting)
- The bot also supports a `!export` command at any time during the session to export the entire **chat history** (including both user inputs and AI responses) into a `.md` file. This makes it easy to revisit recommendations, itinerary details, or planning discussions.
- When exporting the itinerary to a file, user **must** include the format of a file in a filename i.e **travelItinerary.md** or **travelItinerary.pdf**


In [13]:
# Handlers
def handle_get_weather(action, history, memory):
    city = action["location"]
    memory["last_city"] = city
    weather_info = get_weather(city)
    show_response(types.ModelContent(parts=[types.Part.from_text(text=weather_info)]))
    history.append(types.ModelContent(parts=[types.Part.from_text(text=weather_info)]))

def handle_get_tip(action, history, memory):
    query = action["query"]
    city = extract_city_from_text(query)
    memory["last_city"] = city if city else query
    tip_answer = search_and_answer(query)
    show_response(types.ModelContent(parts=[types.Part.from_text(text=tip_answer)]))
    history.append(types.ModelContent(parts=[types.Part.from_text(text=tip_answer)]))

def handle_describe_photo(action, history):
    filename = action["filename"]
    filepath = f"/kaggle/input/photos/{filename}"

    try:
        with open(filepath, "rb") as f:
            f.read(1)
        if filename not in photo_collection.get()['ids']:
            description = describe_photo(filepath)
            add_photo_description_to_chromadb(photo_id=filename, description=description)
            show_response(types.ModelContent(parts=[types.Part.from_text(text=description)]))
            history.append(types.ModelContent(parts=[types.Part.from_text(text=description)]))
        else:
            print(f"🤖 TravelAIAgent: I already have a description for `{filename}`.")
    except FileNotFoundError:
        print(f"🤖 TravelAIAgent: I couldn't find the image `{filename}` in /kaggle/input/photos/.")

def handle_ask_about_photo(user_input, history):
    if not photo_collection.get()['documents']:
        print("🤖 TravelAIAgent: I haven’t analyzed any photos yet. Ask me to describe one first.")
        return
    photo_answer = search_photo_insights(user_input)
    show_response(types.ModelContent(parts=[types.Part.from_text(text=photo_answer)]))
    history.append(types.ModelContent(parts=[types.Part.from_text(text=photo_answer)]))


def handle_image_translate(action, history):
    filename = action["filename"]
    filepath = f"/kaggle/input/photos/{filename}"

    try:
        raw_text = extract_text_from_image(filepath)
        if not raw_text or raw_text.lower() in ["none", ""]:
            print("🤖 TravelAIAgent: I couldn't read any text from the image.")
            return

        translated = translate_to_english(raw_text)
        show_response(types.ModelContent(parts=[types.Part.from_text(text=translated)]))
        history.append(types.ModelContent(parts=[types.Part.from_text(text=translated)]))
    except FileNotFoundError:
        print(f"🤖 TravelAIAgent: I couldn't find `{filename}`. Upload it to /kaggle/input/photos/.")

def handle_get_weather_last(memory, history):
    if memory["last_city"]:
        city = memory["last_city"]
        weather_info = get_weather(city)
        print(f"🤖 TravelAIAgent: Here's the latest weather in {city}:\n{weather_info}")
        history.append(types.ModelContent(parts=[types.Part.from_text(text=weather_info)]))
    else:
        print("🤖 TravelAIAgent: I don't know your last mentioned city yet.")

def handle_get_last_city(memory, history):
    if memory["last_city"]:
        print(f"🤖 TravelAIAgent: The last city you mentioned was {memory['last_city']}.")
        history.append(types.ModelContent(parts=[types.Part.from_text(text=memory['last_city'])]))
    else:
        print("🤖 TravelAIAgent: You haven't mentioned a city yet.")

def handle_get_weather_forecast(action, memory, history):
    city = action["location"]
    memory["last_city"] = city
    forecast = get_weather_summary(city)
    show_response(types.ModelContent(parts=[types.Part.from_text(text=forecast)]))
    history.append(types.ModelContent(parts=[types.Part.from_text(text=forecast)]))

# Get events info
def handle_get_events(action, history, memory):
    city = action.get("location", "Unknown location")
    when = action.get("when", "this week").lower()
    memory["last_city"] = city

    today = datetime.datetime.utcnow()

    # Estimate date range based on natural phrases
    if "tomorrow" in when:
        start_day = today + datetime.timedelta(days=1)
        end_day = start_day
    elif "weekend" in when:
        # Go to next Saturday–Sunday
        days_until_saturday = (5 - today.weekday()) % 7
        start_day = today + datetime.timedelta(days=days_until_saturday)
        end_day = start_day + datetime.timedelta(days=1)
    elif "next week" in when:
        start_day = today + datetime.timedelta(days=7)
        end_day = start_day + datetime.timedelta(days=6)
    elif "next month" in when:
        start_day = today + datetime.timedelta(days=30)
        end_day = start_day + datetime.timedelta(days=7)
    elif "today" in when:
        start_day = today
        end_day = today
    else:
        # Default: this week
        start_day = today
        end_day = today + datetime.timedelta(days=7)

    start = start_day.strftime("%Y-%m-%dT00:00:00Z")
    end = end_day.strftime("%Y-%m-%dT23:59:59Z")

    # Fetch events
    events_info = get_events(city=city, start_date=start, end_date=end)
    show_response(types.ModelContent(parts=[types.Part.from_text(text=events_info)]))
    history.append(types.ModelContent(parts=[types.Part.from_text(text=events_info)]))



# EVALUATE TRAVEL ITINERARY
def evaluate_itinerary(itinerary: str, user_prefs: str) -> str:
    examples = (
        "EXAMPLE 1:\n"
        "ITINERARY:\n"
        "Day 1: Outdoor walking tour during heavy rain\n"
        "Day 2: Multiple museums with little cultural connection to location\n"
        "Day 3: No events or food exploration\n"
        "EVALUATION:\n"
        "⚠️ Travel Score: 5/10. Too much outdoor activity in poor weather. Lack of cultural/local engagement.\n\n"
        "EXAMPLE 2:\n"
        "ITINERARY:\n"
        "Day 1: Market tour + indoor museum during rainy day\n"
        "Day 2: Outdoor gardens during sunshine + local food tour\n"
        "Day 3: Art gallery and live event matching preferences\n"
        "EVALUATION:\n"
        "✅ Travel Score: 9/10. Well-balanced mix of indoor/outdoor. Great alignment with user preferences and local experience.\n\n"
    )

    prompt = (
    "You are a helpful travel assistant who just generated an itinerary for the user — now you want to reflect on it to make sure it’s truly helpful. "
    "Start your reply with a friendly message telling the user that you went ahead and self-evaluated the itinerary you provided, and you're sharing the results. "
    "Briefly explain that this evaluation can help users trust the LLM’s planning and make tweaks if needed.\n\n"
    
    "Then, assess the itinerary based on the following criteria:\n"
    "1. Balance of indoor and outdoor activities (especially for weather conditions)\n"
    "2. Cultural authenticity and local experiences\n"
    "3. Weather suitability\n"
    "4. Personalization based on user preferences\n\n"

    "Provide your response in a friendly tone. In your reply, include:\n"
    "🌍 Travel Score: X/10\n\n"
    "Follow that with a short explanation of the strengths and any gentle suggestions for improvement.\n\n"
    
    f"{examples}"
    f"USER PREFERENCES:\n{user_prefs}\n\n"
    f"ITINERARY:\n{itinerary}"
)


    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )
    return response.text.strip()

# Create and present travel itinerary
def handle_plan_itinerary(action, memory, history):
    city = action.get("location", "Unknown location")
    when = (action.get("when") or "today").lower()
    days = action.get("days")

    # Safe fallback if days is None or not a number
    if not isinstance(days, int):
        if "tomorrow" in when:
            days = 2
        elif "weekend" in when:
            days = 2
        elif "week" in when:
            days = 5
        elif "month" in when:
            days = 7
        else:
            days = 3  # default fallback

    memory["last_city"] = city

    today = datetime.datetime.utcnow()
    if "tomorrow" in when:
        start_day = today + datetime.timedelta(days=1)
    elif "next week" in when:
        start_day = today + datetime.timedelta(days=7)
    elif "next month" in when:
        start_day = today + datetime.timedelta(days=30)
    elif "weekend" in when:
        start_day = today + datetime.timedelta(days=(5 - today.weekday()) % 7)  # next Saturday
    else:
        start_day = today  # default

    # Continue as before
    start_date = start_day.strftime("%Y-%m-%dT00:00:00Z")
    end_day = start_day + datetime.timedelta(days=days)
    end_date = end_day.strftime("%Y-%m-%dT23:59:59Z")

    # Get forecast summary
    forecast = get_weather_summary(city)

    # Get cultural tips
    tips = search_and_answer(f"travel tips for {city}")

    # Get relevant events
    events = get_events(city=city, start_date=start_date, end_date=end_date, max_events=5)

    # Get user preferences
    user_prefs = "\n".join(memory["preferences"]) if memory["preferences"] else "No specific preferences given."

    # Build prompt
    prompt = (
        f"You are a friendly travel planner. Create a detailed {days}-day itinerary for a user visiting {city} starting {when if when else 'today'}.\n"
        f"Use the following information:\n\n"
        f"Weather:\n{forecast}\n\n"
        f"Cultural Tips:\n{tips}\n\n"
        f"Events:\n{events}\n\n"
        f"User Preferences:\n{user_prefs}\n\n"
        f"Suggest fun, practical daily plans. If weather is bad, offer indoor options. Use events when relevant. Make it feel personalized and local."
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    itinerary = response.text.strip()

    # EVALUATION OF ITINERARY
    # 1. Evaluate itinerary with LLM
    evaluation = evaluate_itinerary(itinerary, user_prefs)

    # 2. Show both
    show_response(types.ModelContent(parts=[types.Part.from_text(text=itinerary)]))
    show_response(types.ModelContent(parts=[types.Part.from_text(text=evaluation)]))

    # 3. Save both to history
    history.append(types.ModelContent(parts=[types.Part.from_text(text=itinerary)]))
    history.append(types.ModelContent(parts=[types.Part.from_text(text=evaluation)]))
    
    # Ask for export option
    export_choice = input("\n📝 Export this itinerary? (markdown/pdf/none): ").strip().lower()

    city_slug = city.lower().replace(" ", "_")
    base_filename = f"itinerary_{city_slug}"

    if export_choice in ["markdown", "md"]:
        filename = input(f"📄 Enter filename (default: {base_filename}.md): ").strip() or f"{base_filename}.md"
        try:
            with open(filename, "w", encoding="utf-8") as f:
                f.write(f"# Travel Itinerary for {city.title()}\n\n")
                f.write(itinerary + "\n\n")
                f.write("## ✨ TravelAIAgent Evaluation\n\n")
                f.write(evaluation + "\n")
            print(f"✅ Saved to `{filename}`.")
        except Exception as e:
            print("⚠️ Failed to save .md file:", str(e))

    elif export_choice == "pdf":
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
        from reportlab.lib.pagesizes import LETTER
        from reportlab.lib.styles import getSampleStyleSheet        
        filename = input(f"📄 Enter filename (default: {base_filename}.pdf): ").strip() or f"{base_filename}.pdf"
        try:
            doc = SimpleDocTemplate(filename, pagesize=LETTER, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
            styles = getSampleStyleSheet()
            flowables = []
        
            # Convert itinerary (Markdown-like) into Paragraphs
            for line in itinerary.split('\n'):
                if line.strip() == "":
                    flowables.append(Spacer(1, 12))
                else:
                    flowables.append(Paragraph(line.strip(), styles["Normal"]))

            # Add evaluation section
            # Add evaluation section
            flowables.append(Spacer(1, 24))
            flowables.append(Paragraph("✨ TravelAIAgent Evaluation", styles["Heading2"]))
            for line in evaluation.split('\n'):
                if line.strip() == "":
                    flowables.append(Spacer(1, 12))
                else:
                    flowables.append(Paragraph(line.strip(), styles["Normal"]))

        
            doc.build(flowables)
            print(f"✅ PDF saved as `{filename}`.")
        except Exception as e:
            print("⚠️ Failed to generate PDF:", str(e))

    else:
        print("✅ No problem! Let me know if you want to export later.")

# Export functions
# Export Chat History
def export_chat_history(history, filename="travel_chat_history.md"):
    with open(filename, "w", encoding="utf-8") as f:
        for entry in history:
            if isinstance(entry, types.UserContent):
                f.write("**👤USER:**\n")
            else:
                f.write("**🤖TravelAIAgent:**\n")
            for part in entry.parts:
                f.write(part.text.strip() + "\n\n")
    print(f"✅ Chat history exported to {filename}")

# 🔁 **The Main Chat Loop**
This block of code powers the entire real-time conversation between the user and TravelAIAgent. It’s where everything comes together — intent recognition, memory tracking, handler dispatching, and fallback chat. Here's how it works:

---

**🕘 history and memory Initialization**\
The assistant starts with two critical variables:
1. history: a list that stores the full back-and-forth conversation (in Gemini’s expected format). It keeps context across turns.
2. memory: a dictionary that stores session-specific context like: last_city: the most recently mentioned city (used for follow-ups like "What's the weather like there?"), and  preferences: any user-specified likes (e.g. “I love seafood”) that guide future responses

---
**👋 First Impression: Onboarding Message**\
The assistant prints a warm welcome using the onboarding_message and also adds it to the conversation history. This sets the tone and immediately informs the user of the assistant’s capabilities.

---

**🔁 The Main Loop**\
This is a while True: loop that simulates ongoing conversation. Each time the user enters input: User message is read and cleaned, and intent is interpreted using interpret_user_request()

The assistant routes the request to the appropriate handler: Weather, Cultural tips, Photo analysis, Event search, Itinerary planning, Translation, Memory recall, Handlers take care of fetching data, calling APIs, or prompting Gemini, then store the result back in history.

---

**🧠 Fallback Branch**\
If none of the intent matches are triggered (e.g. the user says something casual or vague): The input is added to history, Gemini is called with full context to continue the conversation naturally. Optionally, if the user shares a preference (e.g. "I love museums"), it’s stored in memory["preferences"] for future personalization. A try/except block ensures any Gemini API failures are handled gracefully without breaking the loop — offering a friendly error instead of a crash.

In [14]:
def start_travel_chat():
    history = []
    memory = {
        "last_city": None,
        "preferences": []
    }
    onboarding_block = types.ModelContent(parts=[types.Part.from_text(text=onboarding_message)])
    history.append(onboarding_block)
    show_response(onboarding_block)
    
    # Main loop
    while True:
        user_input = input("👤 You:").strip()
        
        # Menu options
        # Quit - Use !q, !quit or quit to end the conversation.
        if user_input.strip().lower() in ['!q', 'quit', '!quit']:
            print("Thanks for using TravelAIAgent. Goodbye!")
            break

        # !export - Export chat history
        if user_input.strip().lower() == "!export":
            filename = f"travel_chat_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
            export_chat_history(history, filename)
            continue

        # User actions/requests to LLM
        action = interpret_user_request(user_input)
        if action.get("intent") == "get_weather" and "location" in action:
            handle_get_weather(action, history, memory)
        elif action.get("intent") == "get_tip" and "query" in action:
            handle_get_tip(action, history, memory)
        elif action.get("intent") == "describe_photo" and "filename" in action:
            handle_describe_photo(action, history)
        elif action.get("intent") == "ask_about_photo" and "question" in action:
            handle_ask_about_photo(user_input, history)
        elif action.get("intent") == "image_translate" and "filename" in action:
            handle_image_translate(action, history)
        elif action.get("intent") == "get_weather_last":
            handle_get_weather_last(memory, history)
        elif action.get("intent") == "get_last_city":
            handle_get_last_city(memory, history)
        elif action.get("intent") == "get_weather_forecast" and "location" in action:
            handle_get_weather_forecast(action, memory, history)
        elif action.get("intent") == "plan_itinerary" and "location" in action:
            handle_plan_itinerary(action, memory, history)
        elif action.get("intent") == "get_events" and "location" in action:
            handle_get_events(action, history, memory)
        else:
            if user_input:
                history.append(types.UserContent(parts=[types.Part.from_text(text=user_input)]))

                if any(phrase in user_input.lower() for phrase in ["i like", "i prefer", "i enjoy", "i love", "i want"]):
                    memory["preferences"].append(user_input)

                try:
                    response = client.models.generate_content(
                        model="gemini-2.0-flash",
                        config=config,
                        contents=history
                    )
                    
                    history.append(response.candidates[0].content)
                    show_response(response.candidates[0].content)
                except Exception as e:
                    print("⚠️ TravelAIAgent: I had trouble generating a response. Please try rephrasing.")
                    print("Error:", str(e))

# 💬 **Chat with TravelAIAgent here!**
To begin your travel planning session, click "Run All". Type naturally — like you're talking to a friend — and ask anything about your destination. Type `q` to exit the chat anytime.

In [ ]:
start_travel_chat()

**🤖 TravelAIAgent said:**

👋 Hi there! I'm TravelAIAgent — your AI-powered companion for smarter travel planning 🌍✈️

Here’s what I can help you with:
- 🌦️ Get real-time weather info and multi-day forecasts for any city
- 🧳 Share cultural tips, local etiquette, and safety advice
- 🌐 Translate short phrases or signs into English
- 📸 Read and translate text from uploaded travel photos
- 🏛️ Describe and answer questions about landmarks in your photos
- 🎟️ Find events, concerts, and things to do during your trip
- 🗺️ Build a personalized day-by-day travel itinerary using your preferences, the weather, and local events

🙂 Just talk to me naturally — what's on your mind? What are your travel preferences? Do you have any travel plans soon?

----

👤 You: Hey how is it going?


**🤖 TravelAIAgent said:**

...

----


📝 Export this itinerary? (markdown/pdf/none):  markdown
📄 Enter filename (default: itinerary_san_francisco.md):  itinerary_san_francisco.md


✅ Saved to `itinerary_san_francisco.md`.


👤 You: !export


✅ Chat history exported to travel_chat_20250417_232633.md


## 🪲 Known Bugs
TravelAIAgent is not perfect and can make mistakes. It learns and becomes better day-by-day. One day it'll be very smart!

- "What are some events happening in *** this Sunday" - Can not process "this Sunday" properly and falls back to current day.